In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import os
import json

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def set_seed(seed = 1234):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    torch.backends.cudnn.deterministic = True

    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed()

In [ ]:
def lossCurve(trainLosses,valLosses):
  plt.plot(trainLosses, color="r")
  plt.plot(valLosses, color="g")
  plt.ylabel('Loss')
  plt.xlabel('Epochs')
  plt.legend(['Train Loss','Validation Loss'])
  plt.show()

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.4 MB/s 
     |████████████████████████████████| 6.5 MB 36.0 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
     |████████████████████████████████| 596 kB 42.8 MB/s 
     |████████████████████████████████| 895 kB 38.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json

--2022-03-12 21:07:27--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.110.153, 185.199.111.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M   188MB/s    in 0.2s    

2022-03-12 21:07:27 (188 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]



In [ ]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2022-03-12 21:07:28--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.110.153, 185.199.111.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       100%[===================>]   4.17M  --.-KB/s    in 0.08s   

2022-03-12 21:07:28 (52.8 MB/s) - ‘dev-v2.0.json’ saved [4370528/4370528]



In [ ]:
jsonTrain = "train-v2.0.json"
jsonVal = "dev-v2.0.json"

with open(jsonTrain, 'r') as j:
     trainContents = json.loads(j.read())

with open(jsonVal, 'r') as j:
     valContents = json.loads(j.read())

In [ ]:
def jsonToLists(contents):
  texts=[]
  questions=[]
  answers=[]
  for data in contents['data']:
    for txt in data['paragraphs']: ,
      text = txt['context']
      for qa in txt['qas']: 
        question = qa['question']
        for answer in qa['answers']: 
          texts.append(text)
          questions.append(question)
          answers.append(answer)            
                  
  return texts,questions,answers


In [ ]:
trainTexts,trainQuestions,trainAnswers=jsonToLists(trainContents)
valTexts,valQuestions,valAnswers=jsonToLists(valContents)

In [ ]:
print(len(trainTexts))
print(len(trainQuestions))
print(len(trainAnswers))
print("-----")
print(len(valTexts))
print(len(valQuestions))
print(len(valAnswers))

86821
86821
86821
-----
20302
20302
20302


In [ ]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
      end_idx = answer['answer_start'] + len(answer['text'])

      if context[answer['answer_start']:end_idx] == answer['text']:
        answer['answer_end'] = end_idx
      elif context[answer['answer_start']-1:end_idx-1] == answer['text']:
        answer['answer_start'] = answer['answer_start'] - 1
        answer['answer_end'] = end_idx - 1
      elif context[answer['answer_start']-2:end_idx-2] == answer['text']:
        answer['answer_start'] = answer['answer_start'] - 2
        answer['answer_end'] = end_idx - 2
                
add_end_idx(trainAnswers, trainTexts)
add_end_idx(valAnswers, valTexts)

In [ ]:
from transformers import BertTokenizerFast,DistilBertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')


In [ ]:
trainEncodings=tokenizer(trainTexts,trainQuestions, truncation=True, padding=True, max_length=512)
valEncodings=tokenizer(valTexts,valQuestions, truncation=True, padding=True, max_length=512)

In [ ]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        if(answers[i]['answer_end']==0):
            end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'])) 
        else:
            end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
                   
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(trainEncodings, trainAnswers)
add_token_positions(valEncodings, valAnswers)

In [ ]:
class SquadDataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings

  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

  def __len__(self):
    return len(self.encodings.input_ids)

trainSet = SquadDataset(trainEncodings)
valSet = SquadDataset(valEncodings)

In [ ]:
from transformers import BertForQuestionAnswering,DistilBertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
model.to(device)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 16
trainDataloader = DataLoader(trainSet, batch_size=batch_size, shuffle=True)
valDataloader = DataLoader(valSet, batch_size=batch_size, shuffle=False)

In [ ]:
lr = 1e-5
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

In [ ]:
epochs = 2
trainLosses=[]
valLosses=[]

for i in range(epochs):
  batchLosses = []
  model.train() 

  for batch in trainDataloader:
    
    optimizer.zero_grad()

    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)
    
    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)

    batchLosses.append(outputs[0].item())  

    outputs[0].backward() 

    nn.utils.clip_grad_norm_(model.parameters(),5.0) 

    optimizer.step() 


  trainCurrentLoss = sum(batchLosses)/len(trainDataloader)
  trainLosses.append(trainCurrentLoss) 

 
  with torch.no_grad(): 
    model.eval() 
    batchLosses = []
    for batch in valDataloader:

      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      start_positions = batch['start_positions'].to(device)
      end_positions = batch['end_positions'].to(device)
      
      output = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
      batchLosses.append(output[0].item())

  valCurrentLoss = sum(batchLosses)/len(valDataloader)
  valLosses.append(valCurrentLoss)
  print("-Epoch: {}/{}...".format(i+1, epochs), "Train Loss: {:.6f} |".format(trainCurrentLoss), "Val Loss: {:.6f}".format(valCurrentLoss))

-Epoch: 1/2... Train Loss: 1.581905 | Val Loss: 1.170385
-Epoch: 2/2... Train Loss: 0.968578 | Val Loss: 1.100995


In [ ]:
def predictTheAnswer(text,question,model):
  inputs = tokenizer.encode_plus(question, text, return_tensors='pt',max_length=512, truncation=True).to(device)

  outputs = model(**inputs)
  answer_start = torch.argmax(outputs[0])
  answer_end = torch.argmax(outputs[1]) + 1 

  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
  return answer

In [ ]:
def prediction(contents,model):
  predAnswers={}
  for data in contents['data']:
    for txt in data['paragraphs']: 
      text = txt['context']
      for qa in txt['qas']:
        qid = qa['id']
        question = qa['question']
        predAnswers[qid]=predictTheAnswer(text,question,model)
  return predAnswers

In [ ]:
predAnswers=prediction(valContents,model)

In [ ]:
with open('predictions.txt', 'w') as convert_file:
     convert_file.write(json.dumps(predAnswers))

In [ ]:
!python evaluate-v2.0.py dev-v2.0.json predictions.txt